# Ruby Savon SOAP Client Walkthrough

The `savon` Ruby SOAP Client doesn't provide a command-line interface tool like Python's `zeep` does. So, we'll have to walk through this a bit differently. Let's first start by opening the Ruby REPL tool so we can walk through the WSDL and the available operations.

```bash
export WSDL_URL="http://www.dneonline.com/calculator.asmx?WSDL"
irb
```

Now we can create a client and explore the WSDL.

```ruby
require 'pp'
require 'savon'

client = Savon.client(wsdl: ENV['WSDL_URL'])
=> #<Savon::Client:0x00005632d0292ff8 @globals=#<Savon::GlobalOptions:0x00005632d0292fd0 @option_type=:global, @options={:encoding=>"UTF-8", :soap_version=>1, :namespaces=>{}, :logger=>#<Logger:0x00005632d0292f58 @level=0, @progname=nil, @default_formatter=#<Logger::Formatter:0x00005632d0292ee0 @datetime_format=nil>, @formatter=nil, @logdev=#<Logger::LogDevice:0x00005632d0292e90 @shift_period_suffix=nil, @shift_size=nil, @shift_age=nil, @filename=nil, @dev=#<IO:<STDOUT>>, @mon_mutex=#<Thread::Mutex:0x00005632d0292e18>, @mon_mutex_owner_object_id=47388120356680, @mon_owner=nil, @mon_count=0>>, :log=>false, :filters=>[], :pretty_print_xml=>false, :raise_errors=>true, :strip_namespaces=>true, :delete_namespace_attributes=>false, :convert_response_tags_to=>#<Proc:0x00005632d0292d78@/home/predatorian/.rbenv/versions/2.6.0/lib/ruby/gems/2.6.0/gems/savon-2.12.0/lib/savon/options.rb:86 (lambda)>, :convert_attributes_to=>#<Proc:0x00005632d0292d50@/home/predatorian/.rbenv/versions/2.6.0/lib/ruby/gems/2.6.0/gems/savon-2.12.0/lib/savon/options.rb:87 (lambda)>, :multipart=>false, :adapter=>nil, :use_wsa_headers=>false, :no_message_tag=>false, :follow_redirects=>false, :unwrap=>false, :host=>nil, :wsdl=>"http://www.dneonline.com/calculator.asmx?WSDL"}>, @wsdl=#<Wasabi::Document:0x00005632d0292c38 @document="http://www.dneonline.com/calculator.asmx?WSDL", @adapter=nil, @request=#<HTTPI::Request:0x00005632d0292be8 @follow_redirect=false>>>
```

This imports our libraries we need and creates a base SOAP Client that loaded the WSDL we specified. Now let's see the operations available to the client

```ruby
client.methods
=> [:wsdl, :operations, :call, :operation, :service_name, :build_request, :globals, :to_yaml, :blank?, :pretty_print_cycle, :pretty_print_inspect, :pretty_print_instance_variables, :pretty_print, :instance_variable_defined?, :remove_instance_variable, :instance_of?, :kind_of?, :is_a?, :tap, :instance_variable_get, :instance_variable_set, :instance_variables, :protected_methods, :private_methods, :public_send, :method, :public_method, :singleton_method, :define_singleton_method, :pretty_inspect, :extend, :to_enum, :enum_for, :<=>, :===, :=~, :!~, :eql?, :respond_to?, :freeze, :inspect, :object_id, :send, :to_s, :display, :class, :nil?, :hash, :dup, :singleton_class, :clone, :then, :itself, :yield_self, :untaint, :taint, :tainted?, :trust, :untrust, :untrusted?, :singleton_methods, :frozen?, :methods, :public_methods, :equal?, :!, :==, :instance_exec, :!=, :instance_eval, :__id__, :__send__]
```

Here, we can see that there a few operations available. Generally the ones added to the object are first, then the built-ins that are provided by Ruby are after that. Let's take a look at the `client.wsdl` object.

```ruby
client.wsdl
=> #<Wasabi::Document:0x00005632d0292c38 @document="http://www.dneonline.com/calculator.asmx?WSDL", @adapter=nil, @request=#<HTTPI::Request:0x00005632d0292be8 @follow_redirect=false>>
```

Whoa, what is that? It's an object. We'll need to see what methods are available to that object.

```ruby
client.wsdl.methods
=> [:adapter, :adapter=, :request=, :endpoint=, :element_form_default, :xml, :soap_action, :namespace, :element_form_default=, :soap_action_parameters, :operation_input_parameters, :user_defined, :type_definitions, :type_namespaces, :request, :operations, :xml=, :document?, :soap_actions, :service_name=, :soap_input, :service_name, :parser, :document, :document=, :endpoint, :namespace=, :to_yaml, :blank?, :pretty_print_cycle, :pretty_print_inspect, :pretty_print_instance_variables, :pretty_print, :instance_variable_defined?, :remove_instance_variable, :instance_of?, :kind_of?, :is_a?, :tap, :instance_variable_get, :instance_variable_set, :instance_variables, :protected_methods, :private_methods, :public_send, :method, :public_method, :singleton_method, :define_singleton_method, :pretty_inspect, :extend, :to_enum, :enum_for, :<=>, :===, :=~, :!~, :eql?, :respond_to?, :freeze, :inspect, :object_id, :send, :to_s, :display, :class, :nil?, :hash, :dup, :singleton_class, :clone, :then, :itself, :yield_self, :untaint, :taint, :tainted?, :trust, :untrust, :untrusted?, :singleton_methods, :frozen?, :methods, :public_methods, :equal?, :!, :==, :instance_exec, :!=, :instance_eval, :__id__, :__send__]
```

There are a few new options. However, what we're looking for are the `:operations`. So let's call that function now.

```ruby
client.wsdl.operations
=> {:add=>{:action=>"http://tempuri.org/Add", :input=>"Add", :output=>"AddResponse", :namespace_identifier=>"tns", :parameters=>{:intA=>{:name=>"intA", :type=>"int"}, :intB=>{:name=>"intB", :type=>"int"}}}, :subtract=>{:action=>"http://tempuri.org/Subtract", :input=>"Subtract", :output=>"SubtractResponse", :namespace_identifier=>"tns", :parameters=>{:intA=>{:name=>"intA", :type=>"int"}, :intB=>{:name=>"intB", :type=>"int"}}}, :multiply=>{:action=>"http://tempuri.org/Multiply", :input=>"Multiply", :output=>"MultiplyResponse", :namespace_identifier=>"tns", :parameters=>{:intA=>{:name=>"intA", :type=>"int"}, :intB=>{:name=>"intB", :type=>"int"}}}, :divide=>{:action=>"http://tempuri.org/Divide", :input=>"Divide", :output=>"DivideResponse", :namespace_identifier=>"tns", :parameters=>{:intA=>{:name=>"intA", :type=>"int"}, :intB=>{:name=>"intB", :type=>"int"}}}}
```

Nice, this returned a Hash of our available operations. You can use `pp` to pretty print the Hash to make it easier to read.

```ruby
pp client.wsdl.operations
{:add=>
  {:action=>"http://tempuri.org/Add",
   :input=>"Add",
   :output=>"AddResponse",
   :namespace_identifier=>"tns",
   :parameters=>
    {:intA=>{:name=>"intA", :type=>"int"},
     :intB=>{:name=>"intB", :type=>"int"}}},
 :subtract=>
  {:action=>"http://tempuri.org/Subtract",
   :input=>"Subtract",
   :output=>"SubtractResponse",
   :namespace_identifier=>"tns",
   :parameters=>
    {:intA=>{:name=>"intA", :type=>"int"},
     :intB=>{:name=>"intB", :type=>"int"}}},
 :multiply=>
  {:action=>"http://tempuri.org/Multiply",
   :input=>"Multiply",
   :output=>"MultiplyResponse",
   :namespace_identifier=>"tns",
   :parameters=>
    {:intA=>{:name=>"intA", :type=>"int"},
     :intB=>{:name=>"intB", :type=>"int"}}},
 :divide=>
  {:action=>"http://tempuri.org/Divide",
   :input=>"Divide",
   :output=>"DivideResponse",
   :namespace_identifier=>"tns",
   :parameters=>
    {:intA=>{:name=>"intA", :type=>"int"},
     :intB=>{:name=>"intB", :type=>"int"}}}}
```

We'll want to target the `:add` operation. Now let's actually make a call and then examine the response object.

```ruby
soap_response = client.call(:add, message: {intA: 5, intB: 5})
=> #<Savon::Response:0x00005632cfcb9e60 @http=#<HTTPI::Response:0x00005632cfcba068 @code=200, @headers={"cache-control"=>"private, max-age=0", "content-type"=>"text/xml; charset=utf-8", "vary"=>"Accept-Encoding", "server"=>"Microsoft-IIS/7.5", "x-aspnet-version"=>"2.0.50727", "x-powered-by"=>"ASP.NET", "date"=>"Thu, 28 May 2020 14:29:35 GMT", "content-length"=>"298"}, @raw_body="<?xml version=\"1.0\" encoding=\"utf-8\"?><soap:Envelope xmlns:soap=\"http://schemas.xmlsoap.org/soap/envelope/\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xmlns:xsd=\"http://www.w3.org/2001/XMLSchema\"><soap:Body><AddResponse xmlns=\"http://tempuri.org/\"><AddResult>10</AddResult></AddResponse></soap:Body></soap:Envelope>", @body="<?xml version=\"1.0\" encoding=\"utf-8\"?><soap:Envelope xmlns:soap=\"http://schemas.xmlsoap.org/soap/envelope/\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xmlns:xsd=\"http://www.w3.org/2001/XMLSchema\"><soap:Body><AddResponse xmlns=\"http://tempuri.org/\"><AddResult>10</AddResult></AddResponse></soap:Body></soap:Envelope>">, @globals=#<Savon::GlobalOptions:0x00005632d0292fd0 @option_type=:global, @options={:encoding=>"UTF-8", :soap_version=>1, :namespaces=>{}, :logger=>#<Logger:0x00005632d0292f58 @level=0, @progname=nil, @default_formatter=#<Logger::Formatter:0x00005632d0292ee0 @datetime_format=nil>, @formatter=nil, @logdev=#<Logger::LogDevice:0x00005632d0292e90 @shift_period_suffix=nil, @shift_size=nil, @shift_age=nil, @filename=nil, @dev=#<IO:<STDOUT>>, @mon_mutex=#<Thread::Mutex:0x00005632d0292e18>, @mon_mutex_owner_object_id=47388120356680, @mon_owner=nil, @mon_count=0>>, :log=>false, :filters=>[], :pretty_print_xml=>false, :raise_errors=>true, :strip_namespaces=>true, :delete_namespace_attributes=>false, :convert_response_tags_to=>#<Proc:0x00005632d0292d78@/home/predatorian/.rbenv/versions/2.6.0/lib/ruby/gems/2.6.0/gems/savon-2.12.0/lib/savon/options.rb:86 (lambda)>, :convert_attributes_to=>#<Proc:0x00005632d0292d50@/home/predatorian/.rbenv/versions/2.6.0/lib/ruby/gems/2.6.0/gems/savon-2.12.0/lib/savon/options.rb:87 (lambda)>, :multipart=>false, :adapter=>nil, :use_wsa_headers=>false, :no_message_tag=>false, :follow_redirects=>false, :unwrap=>false, :host=>nil, :wsdl=>"http://www.dneonline.com/calculator.asmx?WSDL", :endpoint=>#<URI::HTTP http://www.dneonline.com/calculator.asmx>}>, @locals=#<Savon::LocalOptions:0x00005632cfca4330 @option_type=:local, @options={:advanced_typecasting=>true, :response_parser=>:nokogiri, :multipart=>false, :message=>{:intA=>5, :intB=>5}, :soap_action=>"http://tempuri.org/Add"}>>
```

This will use the client to call the SOAP service and call the `:add` operation passing it a message of `intA` and `intB` as defined in the operations definition earlier. The SOAP Response we're looking for is mostly kept in the `soap_response.body` object. Let's take a look.

```ruby
soap_response.body
=> {:add_response=>{:add_result=>"10", :@xmlns=>"http://tempuri.org/"}}
```

Sweet! Seems pretty simple from here. Now we can call it just as though it were a Hash.

```ruby
soap_response.body[:add_response][:add_result]
=> "10"
```

There are plenty of other options to provide flexibility and control over your Savon SOAP operations. 

# Ruby Code

This is an extremely stripped down version of the code we did above in the `irb` REPL tool.

```ruby
#!/usr/bin/env ruby
# frozen_string_literal: true

require 'pp'
require 'savon'

client = Savon.client(wsdl: 'http://www.dneonline.com/calculator.asmx?WSDL')

pp client.wsdl.operations

soap_response = client.call(:add, message: { intA: 5, intB: 5 })

raise "The value is not 10" unless soap_response.body[:add_response][:add_result] == 10

puts soap_response.body[:add_response][:add_result]
```

# References

* [Savonrb](https://rubygems.org/gems/savon), [Docs](http://savonrb.com/)
* [Savon Guide, way better than the actual docs](https://fagiani.github.io/savon/)